# Birge and Louveaux’s Farmer Problem

Birge and Louveaux [BirgeLouveauxBook] make use of the example of a farmer who has 500 acres that can be planted in wheat, corn or sugar beets, at a per acre cost of 150, 230 and 260 (Euros, presumably), respectively. The farmer needs to have at least 200 tons of wheat and 240 tons of corn to use as feed, but if enough is not grown, those crops can be purchased for 238 and 210, respectively. Corn and wheat grown in excess of the feed requirements can be sold for 170 and 150, respectively. A price of 36 per ton is guaranteed for the first 6000 tons grown by any farmer, but beets in excess of that are sold for 10 per ton. The yield is 2.5, 3, and 20 tons per acre for wheat, corn and sugar beets, respectively. (description taken from https://pysp.readthedocs.io/en/latest/pysp.html#birge-and-louveaux-s-farmer-problem)

![Farmers data](farmers_data.png "")

## Abstract model definition

We define the abastract model containig the specification of the model in a parametric form

In [1]:
import pyomo.environ as pyo

In [3]:
from pyomo.environ import *
#-------------------------abastract model----------------------------
model = pyo.AbstractModel()

#---------------------------index sets-------------------------------
model.crops = Set()

#---------------------------parameters-------------------------------
#objective function
model.plantingCosts = Param(model.crops, within = PositiveReals)
model.purchasePrices = Param(model.crops, within = PositiveReals)
model.sellingPricesSubQuota = Param(model.crops, within = PositiveReals)

def sellingQuotaPricesValidation(model, value, i):
    return model.sellingPricesSubQuota[i] >= model.sellingPricesOverQuota[i]

model.sellingPricesOverQuota = Param(model.crops, validate = sellingQuotaPricesValidation)

#constraints
model.totalArea = Param(within = PositiveReals)
model.productionRequirement = Param(model.crops, within = NonNegativeReals)
model.pricesQuota = Param(model.crops, within = PositiveReals)
model.cropsYielding = Param(model.crops, within = NonNegativeReals)

#---------------------------variables--------------------------------
model.acresToCrops = Var(model.crops, bounds = (0.0, model.totalArea))
model.cropsPurchased = Var(model.crops, bounds = (0.0, None))
model.cropsSoldedSubQuota = Var(model.crops, bounds = (0.0, None))
model.cropsSoldedOverQuota = Var(model.crops, bounds = (0.0, None))

#--------------------------constraints-------------------------------
def totalAreaConstraint(model):
    return summation(model.acresToCrops) <= model.totalArea

def productionRequirementConstraint(model, i):
    return (model.cropsYielding[i]*model.acresToCrops[i]) + model.cropsPurchased[i]\
         - model.cropsSoldedSubQuota[i] - model.cropsSoldedOverQuota[i] >= model.productionRequirement[i]

def quotaConstraint(model, i):
    return model.cropsSoldedSubQuota[i] <= model.pricesQuota[i]

model.totalAreaConstraint = Constraint(rule = totalAreaConstraint)
model.productionRequirementConstraint = Constraint(model.crops, rule = productionRequirementConstraint)
model.quotaConstraint = Constraint(model.crops, rule = quotaConstraint)

#-----------------------objective function---------------------------
#first stage
def firstStageCost(model):
    return summation(model.plantingCosts, model.acresToCrops)

model.firstStageCost = Expression(rule = firstStageCost)

#second stage
def secondStageCost(model):
    e = summation(model.purchasePrices, model.cropsPurchased)
    e -= summation(model.sellingPricesSubQuota, model.cropsSoldedSubQuota)
    e -= summation(model.sellingPricesOverQuota, model.cropsSoldedOverQuota)
    return e

model.secondStageCost = Expression(rule = secondStageCost)

#total
def totalCost(model):
    return model.firstStageCost + model.secondStageCost

model.totalCost = Objective(rule = totalCost, sense = minimize)

## Data initialization

We defined data necessary to initialize the abastract model

In [5]:
data = {None: {
    'crops': {None: ['wheat', 'corn', 'sugar beets']}, 
    'plantingCosts': {'wheat': 150, 'corn': 230, 'sugar beets': 260}, 
    'purchasePrices': {'wheat': 238, 'corn': 210, 'sugar beets': 100000},
    'sellingPricesSubQuota': {'wheat': 170, 'corn': 150, 'sugar beets': 36},
    'sellingPricesOverQuota': {'wheat': 170, 'corn': 150, 'sugar beets': 10},
    'totalArea': {None: 500},
    'productionRequirement': {'wheat': 200, 'corn': 240, 'sugar beets': 0},
    'pricesQuota': {'wheat': 100000, 'corn': 100000, 'sugar beets': 6000},
    'cropsYielding': {'wheat': 2.5, 'corn': 3, 'sugar beets': 20}
}
}

## Istance creation

In [6]:
istance = model.create_instance(data=data)
istance.pprint()

1 Set Declarations
    crops : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'wheat', 'corn', 'sugar beets'}

8 Param Declarations
    cropsYielding : Size=3, Index=crops, Domain=NonNegativeReals, Default=None, Mutable=False
        Key         : Value
               corn :     3
        sugar beets :    20
              wheat :   2.5
    plantingCosts : Size=3, Index=crops, Domain=PositiveReals, Default=None, Mutable=False
        Key         : Value
               corn :   230
        sugar beets :   260
              wheat :   150
    pricesQuota : Size=3, Index=crops, Domain=PositiveReals, Default=None, Mutable=False
        Key         : Value
               corn : 100000
        sugar beets :   6000
              wheat : 100000
    productionRequirement : Size=3, Index=crops, Domain=NonNegativeReals, Default=None, Mutable=False
        Key         : Value
               corn :   240
        sugar beets

## Deterministic problem solution

We solve the model using the cplex solver

In [8]:
opt = SolverFactory('cplex_direct')
opt.solve(istance)

{'Problem': [{'Name': '', 'Lower bound': -118600.0, 'Upper bound': -118600.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 12, 'Number of nonzeros': None, 'Sense': 1}], 'Solver': [{'Name': 'CPLEX 22.1.0.0', 'Status': 'ok', 'Wallclock time': 0.00099945068359375, 'Termination condition': 'optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

We can analize results using the display methods

In [9]:
istance.display()

Model unknown

  Variables:
    acresToCrops : Size=3, Index=crops
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
               corn :   0.0 :  80.0 :   500 : False : False :  Reals
        sugar beets :   0.0 : 300.0 :   500 : False : False :  Reals
              wheat :   0.0 : 120.0 :   500 : False : False :  Reals
    cropsPurchased : Size=3, Index=crops
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
               corn :   0.0 :   0.0 :  None : False : False :  Reals
        sugar beets :   0.0 :   0.0 :  None : False : False :  Reals
              wheat :   0.0 :   0.0 :  None : False : False :  Reals
    cropsSoldedSubQuota : Size=3, Index=crops
        Key         : Lower : Value  : Upper : Fixed : Stale : Domain
               corn :   0.0 :    0.0 :  None : False : False :  Reals
        sugar beets :   0.0 : 6000.0 :  None : False : False :  Reals
              wheat :   0.0 :    0.0 :  None : False : False :  Reals
    cropsSolde

## Stochastic problem solution

In order to use mpi-sppy library to solve the stochastic problem, we define a function that take as argument the value of parameters that change over different scenarios and build an istance of the abstract model

In [10]:
def build_model(yields):
    data = {None: {
    'crops': {None: ['wheat', 'corn', 'sugar beets']}, 
    'plantingCosts': {'wheat': 150, 'corn': 230, 'sugar beets': 260}, 
    'purchasePrices': {'wheat': 238, 'corn': 210, 'sugar beets': 100000},
    'sellingPricesSubQuota': {'wheat': 170, 'corn': 150, 'sugar beets': 36},
    'sellingPricesOverQuota': {'wheat': 170, 'corn': 150, 'sugar beets': 10},
    'totalArea': {None: 500},
    'productionRequirement': {'wheat': 200, 'corn': 240, 'sugar beets': 0},
    'pricesQuota': {'wheat': 100000, 'corn': 100000, 'sugar beets': 6000},
    'cropsYielding': {'wheat': yields['wheat'], 'corn': yields['corn'], 'sugar beets': yields['sugar beets']}
    }}

    istance = model.create_instance(data=data)
    return istance

We assume, as in the original problem, three different scenarios. Then, we create a function able to generate those scenarios

In [11]:
import mpisppy.utils.sputils as sputils

def scenario_creator(scenario_name):
    if scenario_name == "good":
        yields = {'wheat': 3, 'corn': 3.6, 'sugar beets': 24}
    elif scenario_name == "average":
        yields = {'wheat': 2.5, 'corn': 3, 'sugar beets': 20}
    elif scenario_name == "bad":
        yields = {'wheat': 2, 'corn': 2.4, 'sugar beets': 16}
    else:
        raise ValueError("Unrecognized scenario name")

    istance= build_model(yields)

    #we specify which part of the objective function and which set of variables
    #belong to the first stage of the problem
    sputils.attach_root_node(istance, istance.plantingCosts, [istance.acresToCrops])
    istance._mpisppy_probability = 1.0 / 3
    return istance

[    0.00] Initializing mpi-sppy


### Solving the extensive form

In [12]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "cplex_direct"}
all_scenario_names = ["good", "average", "bad"]
ef = ExtensiveForm(options, all_scenario_names, scenario_creator)
results = ef.solve_extensive_form()

objval = ef.get_objective_value()
print(objval)

soln = ef.get_root_solution()
for (var_name, var_val) in soln.items():
    print(var_name, var_val)

[    3.39] Initializing SPBase
-108389.99999999999
acresToCrops[corn] 80.0
acresToCrops[sugar beets] 250.0
acresToCrops[wheat] 170.0


### Resolving using PH (Progressive Hedging)

In [42]:
from mpisppy.opt.ph import PH

options = {
    "solvername": "cplex_persistent",
    "PHIterLimit": 200,
    "defaultPHrho": 10,
    "convthresh": 1e-9,
    "verbose": False,
    "display_progress": False,
    "display_timing": False,
    "iter0_solver_options": dict(),
    "iterk_solver_options": dict(),
}
all_scenario_names = ["good", "average", "bad"]
ph = PH(
    options,
    all_scenario_names,
    scenario_creator,
)

[14976.23] Initializing SPBase
[14976.26] Initializing PHBase


In [43]:
ph.ph_main()

[14979.00] Creating solvers
[14979.03] Entering solve loop in PHBase.Iter0
[14987.02] Reached user-specified limit=200 on number of PH iterations


(6.260371318628838e-07, -108389.99946897829, -115405.55555555552)

In [44]:
ph.report_var_values_at_rank0()

                          |  average      bad        good    
acresToCrops[corn]        |    80.0000    80.0000    80.0000 
acresToCrops[sugar beets] |   250.0000   250.0000   250.0000 
acresToCrops[wheat]       |   170.0000   170.0000   170.0000 


# Solution evaluation

We can evaluate the solution of a stochastic program problem by comparing it with other relevant quantities. In particular, we define:

- **WS** (wait and see) solution as: 

$$ WS = E_{\xi}[min_{x}z(x,\xi)] $$

- **RP** (recourse procedure) solution as:

$$ RP = min_{x}E_{\xi}z(x,\xi) $$

- **EV** (expected value problem) solution as:

$$ EV = min_{x}E_{\xi}z(x, \bar{\xi}) $$

- **EEV** (expectation of the expected solutions) as:

$$ E_{\xi}[z(\bar{x}(\bar{\xi}),\xi)] $$

with $ \bar{x}(\bar{\xi}) $ is a solution of $ min_{x}E_{\xi}z(x, \bar{\xi}) $ 

From the previous one can define:

- **EVPI** (expected value of perfect information) as:

$$ EVPI = RP - WS $$

- **VSS** (value of stochastic solution) as:

$$ VSS = EEV - RP $$

The value of the **RP** solution has been previously compute

In [109]:
RP = ph.Eobjective()
print("RP : " + str(round(RP,3)))

RP : -108389.999


In order to determine the value of the **WS** solution we have to solve a deterministic problem for each scenario (the idea is that we know beforehand what returns will be) and then take the mean value. We recall that the three scenarios are:

|| Wheat (T)   | Corn (T)     | Sugar beets (T)  |
|---| :---------: | :--------: | :-------------: |
|good|     3     |   3.6    |       24      |
|average|    2.5    |    3     |       20      |
|bad|     2     |   2.4    |       16      |

We choose the solver

In [110]:
opt = SolverFactory('cplex_direct')

We solve the problem in the three different scenarios

In [111]:
istanceGood = build_model({'wheat': 3, 'corn': 3.6, 'sugar beets': 24})
opt.solve(istanceGood)

{'Problem': [{'Name': '', 'Lower bound': -167666.66666666666, 'Upper bound': -167666.66666666666, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 12, 'Number of nonzeros': None, 'Sense': 1}], 'Solver': [{'Name': 'CPLEX 22.1.0.0', 'Status': 'ok', 'Wallclock time': 0.0009996891021728516, 'Termination condition': 'optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [112]:
WSGood = istanceGood.totalCost()

In [113]:
istanceAverage = build_model({'wheat': 2.5, 'corn': 3, 'sugar beets': 20})
opt.solve(istanceAverage)

{'Problem': [{'Name': '', 'Lower bound': -118600.0, 'Upper bound': -118600.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 12, 'Number of nonzeros': None, 'Sense': 1}], 'Solver': [{'Name': 'CPLEX 22.1.0.0', 'Status': 'ok', 'Wallclock time': 0.000985860824584961, 'Termination condition': 'optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [114]:
WSAverage = istanceAverage.totalCost()

In [117]:
istanceBad = build_model({'wheat': 2, 'corn': 2.4, 'sugar beets': 16})
opt.solve(istanceBad)

{'Problem': [{'Name': '', 'Lower bound': -59950.0, 'Upper bound': -59950.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 12, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 12, 'Number of nonzeros': None, 'Sense': 1}], 'Solver': [{'Name': 'CPLEX 22.1.0.0', 'Status': 'ok', 'Wallclock time': 0.0010292530059814453, 'Termination condition': 'optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [118]:
WSBad = istanceBad.totalCost()

So we can compute the **WS** solution (*notice that is the same computed by the solver using the PH approach*) 

In [119]:
import numpy as np

WS = np.mean([WSGood, WSAverage, WSBad])
print("WS : " + str(round(WS,3)))

WS : -115405.556


So **the value of the perfect information** would be:

In [122]:
EVPI = RP - WS
print("EVPI: " + str(EVPI)) 

EVPI: 7015.556086577271
